In [27]:
import pytds
import re 
import spacy
import pandas as pd
from time import time
connect_string = ""
try:
    conn = pytds.connect(server = '34.69.166.47', database = 'TBDraft', user = 'sqlserver', password='TBuilder')
    print("{c} is working".format(c=connect_string))
except pytds.Error as ex:
    print("{c} is not working".format(c=connect_string))

 is working


In [63]:
from tika import parser
import os
df = pd.DataFrame(columns=['CandidateID','Keyword', 'Resume', 'Cleaned_Resume'])
for dirname , _, filenames in os.walk('C:/Users/maxbr/ECEN 403/ECEN-403/Test Resumes'):
    for filename in filenames:
       path = os.path.join(dirname, filename)
       raw = parser.from_file(path)
       df = df.append({'CandidateID' : filename[:-4],'Keyword' : dirname.replace("C:/Users/maxbr/ECEN 403/ECEN-403/Test Resumes\\", ""), 'Resume' : " ".join(raw['content'].strip().split('\n')[1:]), 'Cleaned_Resume' : ""}, 
                ignore_index = True)
   
df.shape

def cleanResume(resumeText):
    resumeText = re.sub('http\S+\s*', ' ', resumeText)  # remove URLs
    resumeText = re.sub('RT|cc', ' ', resumeText)  # remove RT and cc
    resumeText = re.sub('#\S+', '', resumeText)  # remove hashtags
    resumeText = re.sub('@\S+', '  ', resumeText)  # remove mentions
    resumeText = re.sub('[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""), ' ', resumeText)  # remove punctuations
    resumeText = re.sub(r'[^\x00-\x7f]',r' ', resumeText) 
    resumeText = re.sub('\s+', ' ', resumeText)  # remove extra whitespace
    return resumeText
df['Cleaned_Resume'] = df.Resume.apply(lambda x: cleanResume(x))
nlp = spacy.load('en_core_web_sm', disable=['ner', 'parser']) # disabling Named Entity Recognition for speed

def cleaning(doc):
    # Lemmatizes and removes stopwords
    # doc needs to be a spacy Doc object
    txt = [token.lemma_ for token in doc if not token.is_stop]
    # Word2Vec uses context words to learn the vector representation of a target word,
    # if a sentence is only one or two words long,
    # the benefit for the training is very small
    if len(txt) > 2:
        return ' '.join(txt)

brief_cleaning = (re.sub("[^A-Za-z']+", ' ', str(row)).lower() for row in df['Cleaned_Resume'])

t = time()

txt = [cleaning(doc) for doc in nlp.pipe(brief_cleaning, batch_size=5000)]

print('Time to clean up everything: {} mins'.format(round((time() - t) / 60, 2)))
df['Cleaned_Resume'] = txt
df.head()

Time to clean up everything: 0.16 mins


,CandidateID,Keyword,Resume,Cleaned_Resume
0,Andres-Salazar,Electronics_Engineer,"Andres Salazar Fort Worth, TX -Email me on I...",andre salazar fort worth tx email talent ele...
1,Arturo-Cervantes,Electronics_Engineer,"Arturo Cervantes Mission, TX -Email me on In...",arturo cervante mission tx email seek employ...
2,Brandon-Karnoski,Electronics_Engineer,"Brandon Karnoski League City, TX -Email me o...",brandon karnoski league city tx email work e...
3,Charles-Sampson,Electronics_Engineer,Charles Sampson Independent Contractor - Up-...,charles sampson independent contractor rev i...
4,Clay-Styer,Electronics_Engineer,"Clay Styer Woodway, TX -Email me on Indeed: ...",clay styer woodway tx email experience profe...


In [64]:
cursor = conn.cursor()
cursor.execute('''TRUNCATE TABLE TrainingResumes''')
conn.commit()
#cursor.execute("INSERT INTO HumanResources.DepartmentTest (DepartmentID,Name,GroupName) values(?,?,?)", row.DepartmentID, row.Name, row.GroupName)
insert_q = "INSERT INTO TrainingResumes (keyword, dataText) "
for i in range(len(df['Cleaned_Resume'])):
    keyword = df.Keyword[i]
    data = df.Cleaned_Resume[i]

    values = "Values('" + keyword + "','" + data + "')"
    commd = insert_q + values
    
    cursor.execute(commd)
    
conn.commit()

In [65]:
df_training = pd.read_sql_query("SELECT ALL * FROM TrainingResumes", conn)
conn.close()
df_training

,keyword,dataText
0,Electronics_Engineer,andre salazar fort worth tx email talent ele...
1,Electronics_Engineer,arturo cervante mission tx email seek employ...
2,Electronics_Engineer,juan avila charleston sc email experience el...
3,Electronics_Engineer,travis ray engineer product designer develop...
4,Embedded_Systems,daniel koh forney tx email software developm...
...,...,...
101,Software_Engineer,kai schmidt software engineer citrus height ...
102,Software_Engineer,nathan nguyen irvine email work experience s...
103,Software_Engineer,peter pagano bohemia ny email authorize work...
104,Software_Engineer,steven nichol c c software engineer grand ra...


In [2]:
import pytds
connect_string = ""
try:
    conn = pytds.connect(server = '34.69.166.47', database = 'TBDraft', user = 'sqlserver', password='TBuilder')
    print("{c} is working".format(c=connect_string))
except pytds.Error as ex:
    print("{c} is not working".format(c=connect_string))

 is working


In [37]:
df_roles = pd.read_sql_query("SELECT ALL * FROM Roles", conn)
df_projects = pd.read_sql_query("SELECT ALL * FROM Project", conn)

print(df_projects)
print(df_roles)

   projectName                                             Descrp  \
0  Teambuilder  Develop a system that uses student resumes and...   

  ProfessorName  
0   Kevin Nowka  
        projectName roleName            roleDescription  Candidate1  \
0  Solar Skateboard   Role 1      Power Supply Desginer           0   
1  Solar Skateboard   Role 2  Microcontroller Subsystem           0   
2  Solar Skateboard   Role 3        Wireless Controller           0   
3         Music App   Role 1              App Developer           0   
4         Music App   Role 2  Microcontroller/Bluetooth           0   
5         Music App   Role 3         Audio Transmission           0   
6         Music App   Role 4        Power Supply Design           0   

   Candidate2  Candidate3                             proSkills  \
0           0           0           Power Supply, DC Converters   
1           0           0                           Python, C++   
2           0           0                 Wireless, ADC

In [36]:
cursor = conn.cursor()
insert_q = "INSERT INTO Roles (projectName, roleName, roleDescription, Candidate1, Candidate2, Candidate3, proSkills, conSkills ) "
values = "Values('Music App', 'Role 4', 'Power Supply Design', '' , '' , '', 'Power Supply, DC Converter', '')"
commd = insert_q + values   
cursor.execute(commd)    
conn.commit()

In [18]:
cursor.execute('''TRUNCATE TABLE Roles''')
conn.commit()

In [25]:
conn.close()